# AI-PSCI-005: Compound Data Acquisition from ChEMBL
**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 3 | Module: AI in Drug Discovery | Estimated Time: 60-90 minutes**

**Prerequisites**: AI-PSCI-001, AI-PSCI-002, AI-PSCI-003, AI-PSCI-004

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Query the ChEMBL database programmatically using Python
2. Filter bioactivity data by target, activity type, and quality metrics
3. Convert IC50 values to pIC50 for easier comparison
4. Create clean datasets ready for downstream analysis
5. Understand the structure and content of ChEMBL data

---

## 📚 Background

### What is ChEMBL?

**ChEMBL** is the world's largest open-access database of bioactive molecules with drug-like properties. Maintained by the European Bioinformatics Institute (EMBL-EBI), it contains:

- **>2 million** distinct compounds
- **>16 million** bioactivity measurements
- Data for **>14,000** drug targets
- Information from scientific literature, PubChem, and other sources

ChEMBL is the starting point for most computational drug discovery projects because it provides experimentally measured activity data linking compounds to biological targets.

### Why Programmatic Access Matters

While ChEMBL has a web interface, programmatic access via the `chembl_webresource_client` Python library enables:
- **Reproducible** data retrieval
- **Automated** filtering and preprocessing
- **Integration** into computational pipelines
- **Batch** processing of multiple queries

### Key Concepts

- **ChEMBL ID**: Unique identifier (e.g., CHEMBL202 for DHFR target)
- **IC50**: Half-maximal inhibitory concentration (nM) — lower is more potent
- **pIC50**: -log₁₀(IC50 in M) — higher is more potent, easier to compare
- **Bioactivity type**: IC50, Ki, EC50, etc.
- **Assay type**: B (binding), F (functional), A (ADMET)

---

## 🛠️ Setup

Run this cell to install required packages:

In [ ]:
#@title 🛠️ Install Packages
!pip install chembl_webresource_client rdkit -q
print("✅ Packages installed successfully!")

Import the required libraries:

In [ ]:
#@title 📦 Import Libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Draw

# Set display options
pd.set_option('display.max_columns', 10)

print("✅ All libraries imported!")

---

## 🔬 Guided Inquiry 1: Connecting to ChEMBL and Understanding the API

### Context

The ChEMBL web resource client provides a Pythonic interface to the ChEMBL database. It uses "lazy evaluation" — queries aren't executed until you actually need the data, reducing network traffic and speeding up your work.

The main resources (API endpoints) we'll use are:
- **target**: Information about drug targets (proteins, organisms)
- **activity**: Bioactivity measurements (IC50, Ki, etc.)
- **molecule**: Compound structures and properties

### Your Task

Using your AI assistant, write code to:

1. Create API client objects for targets, activities, and molecules
2. Search for a target by name (e.g., "DHFR" or "dihydrofolate reductase")
3. Display the first few results to understand the data structure

💡 **Prompting Tips**:
- Ask: "How do I use the chembl_webresource_client to search for targets?"
- Ask: "What fields are available when querying ChEMBL targets?"
- If results seem empty, try broader search terms

### Verification

After running your code, confirm:
- [ ] You can create the three API client objects
- [ ] Target search returns results (check for CHEMBL202 for DHFR)
- [ ] You understand the difference between target_type values

📓 **Lab Notebook**: Record what fields are available in the target search results.

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 2: Retrieving Target Information

### Context

Now that we can search for targets, let's retrieve detailed information for specific targets. In drug discovery, you typically start with a target of interest identified by:
- **ChEMBL ID**: Direct database identifier (e.g., CHEMBL202)
- **UniProt ID**: Protein database identifier (e.g., P0ABQ4)

Let's explore several drug targets that represent different therapeutic areas.

### Your Task

Using your AI assistant, write code to:

1. Define a dictionary of 3-4 targets with their ChEMBL IDs:
   - DHFR: CHEMBL202 (antibiotics)
   - COX-2: CHEMBL230 (inflammation)
   - AChE: CHEMBL220 (Alzheimer's)

2. Query ChEMBL for each target's full information

3. Create a summary table showing target name, organism, and type

💡 **Prompting Tips**:
- Ask: "How do I get target information using a ChEMBL ID?"
- Ask: "What's the difference between .search() and .filter()?"
- Try using `target_chembl_id` as a filter parameter

### Verification

After running your code, confirm:
- [ ] All targets are found (no empty results)
- [ ] DHFR (CHEMBL202) is human dihydrofolate reductase
- [ ] COX-2 and AChE are human proteins

📓 **Lab Notebook**: Note which organisms each target comes from. Why might E. coli DHFR be used instead of human DHFR?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 3: Fetching Bioactivity Data

### Context

The heart of ChEMBL is its bioactivity data — millions of measurements linking compounds to targets. For drug discovery, we typically want:

- **IC50 values**: How much compound is needed to inhibit 50% of target activity
- **Binding assays (type B)**: Direct measurement of compound-target interaction
- **Exact values (relation '=')**: Not ranges like ">1000"

Let's query bioactivity data for one of our targets.

### Your Task

Using your AI assistant, write code to:

1. Query bioactivity data for DHFR (CHEMBL202)
2. Filter for:
   - Activity type: IC50
   - Assay type: B (binding)
   - Relation: '=' (exact values only)
3. Display the first 10 results and the total count
4. Examine the data structure (what columns are available?)

💡 **Prompting Tips**:
- Ask: "How do I filter ChEMBL activities by IC50 and assay type?"
- Ask: "What fields does the activity API return?"
- This query may take 1-2 minutes for large targets

### Verification

After running your code, confirm:
- [ ] Query returns hundreds of results for DHFR
- [ ] Results include molecule_chembl_id, standard_value, standard_units
- [ ] Units are in nM (nanomolar)

📓 **Lab Notebook**: How many IC50 measurements are available for DHFR? How does this compare to the other targets?

In [ ]:
# Your code here



In [ ]:
# Your code here (optional: explore the data further)



---

## 🔬 Guided Inquiry 4: Converting IC50 to pIC50

### Context

IC50 values span many orders of magnitude (from 0.001 nM to 1,000,000 nM). This makes them hard to compare and visualize. The **pIC50** transformation solves this:

$$pIC50 = -\log_{10}(IC50_{M})$$

Where IC50 is in molar units. For nM values:

$$pIC50 = 9 - \log_{10}(IC50_{nM})$$

This gives us:
- **Higher pIC50 = more potent** (easier to interpret)
- Values typically range from 4 to 10
- Each unit = 10-fold difference in potency

### Your Task

Using your AI assistant, write code to:

1. Write a function to convert IC50 (nM) to pIC50
2. Apply it to your bioactivity DataFrame
3. Create a histogram showing the pIC50 distribution
4. Identify the most potent compounds (highest pIC50)

💡 **Prompting Tips**:
- Ask: "How do I convert IC50 in nM to pIC50?"
- Ask: "How do I apply a function to a pandas DataFrame column?"
- Watch out for zero or negative values (log undefined!)

### Verification

After running your code, confirm:
- [ ] pIC50 of 100 nM = 7.0
- [ ] pIC50 of 10 nM = 8.0
- [ ] pIC50 of 1 nM = 9.0
- [ ] Most potent DHFR inhibitors have pIC50 > 8

📓 **Lab Notebook**: What is the pIC50 of Trimethoprim (IC50 ≈ 5 nM for E. coli DHFR)?

In [ ]:
# Your code here



In [ ]:
# Your code here (apply to DataFrame and visualize)



---

## 🔬 Guided Inquiry 5: Retrieving Compound Structures

### Context

Now we have bioactivity data, but we also need the molecular structures (SMILES) for downstream analysis. ChEMBL stores structures in the `molecule` resource, linked by `molecule_chembl_id`.

### Your Task

Using your AI assistant, write code to:

1. Get the list of unique molecule ChEMBL IDs from your bioactivity data
2. Query ChEMBL for the structures (SMILES) of these molecules
3. Merge the structure data with the bioactivity data
4. Display some of the most potent compounds with their structures

💡 **Prompting Tips**:
- Ask: "How do I query ChEMBL for molecule structures?"
- Ask: "How do I merge two DataFrames on a common column?"
- Large queries may need to be batched

### Verification

After running your code, confirm:
- [ ] Most molecules have valid SMILES strings
- [ ] Merged DataFrame has both pIC50 and SMILES
- [ ] Can visualize top compounds with RDKit

📓 **Lab Notebook**: How many unique compounds have IC50 data for DHFR? What fraction have valid structures?

In [ ]:
# Your code here



In [ ]:
# Your code here (merge and visualize)



In [ ]:
# Your code here (visualize top compounds)



---

## 🔬 Guided Inquiry 6: Creating a Clean Dataset

### Context

Before using ChEMBL data for machine learning or virtual screening, we need to clean it:
1. Remove duplicates (same molecule tested multiple times)
2. Handle outliers (unrealistic values)
3. Ensure consistent units
4. Select relevant columns

### Your Task

Using your AI assistant, write code to:

1. Remove duplicate molecules (keep one with median IC50)
2. Filter to keep only values in nM units
3. Remove outliers (pIC50 < 3 or > 12)
4. Create a final clean DataFrame with: ChEMBL ID, SMILES, IC50, pIC50
5. Save the dataset to a CSV file

💡 **Prompting Tips**:
- Ask: "How do I handle duplicate entries in pandas?"
- Ask: "What's a reasonable pIC50 range for drug-like compounds?"
- Consider using `.groupby()` for aggregating duplicates

### Verification

After running your code, confirm:
- [ ] No duplicate molecule ChEMBL IDs
- [ ] pIC50 values are in reasonable range (3-12)
- [ ] CSV file is created successfully
- [ ] Dataset is ready for downstream analysis

📓 **Lab Notebook**: How many compounds were removed during cleaning? What were the main reasons?

In [ ]:
# Your code here



In [ ]:
# Your code here (create final dataset)



In [ ]:
# Your code here (save and visualize)



---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Connect to ChEMBL and use the API client objects
- [ ] Search for targets and retrieve target information
- [ ] Query bioactivity data with appropriate filters
- [ ] Convert IC50 to pIC50 and interpret the values
- [ ] Retrieve compound structures (SMILES) from ChEMBL
- [ ] Clean and prepare a dataset for downstream analysis

### Your lab notebook should include:

- [ ] List of ChEMBL IDs for targets of interest
- [ ] Summary of bioactivity data available for each target
- [ ] pIC50 distribution plot for your target
- [ ] Clean dataset saved as CSV
- [ ] Notes on data quality and cleaning decisions

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Data Quality**: ChEMBL data comes from many different labs using different assay protocols. How might this affect your analysis? What could you do to address this?

2. **Target Selection**: DHFR exists in both bacteria and humans. Why did we use CHEMBL202 (human DHFR) instead of bacterial DHFR? What would change if we were studying antibiotics?

3. **Potency Interpretation**: The most potent DHFR inhibitor in ChEMBL has pIC50 ~10. Is this compound a good drug candidate? What other factors besides potency matter?

---

## 📖 Further Reading

- [ChEMBL Database](https://www.ebi.ac.uk/chembl/) - Web interface for browsing
- [ChEMBL Web Resource Client Documentation](https://github.com/chembl/chembl_webresource_client) - API documentation
- [Gaulton et al. (2017)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5210557/) - ChEMBL database paper
- [TeachOpenCADD T001](https://projects.volkamerlab.org/teachopencadd/talktorials/T001_query_chembl.html) - Original ChEMBL tutorial

---

## 🔗 Connection to Research

The ChEMBL data acquisition skills you learned today are the foundation of modern drug discovery:

- **Virtual screening**: Use ChEMBL compounds as training data for machine learning models
- **Lead identification**: Find starting points for drug optimization
- **SAR analysis**: Understand structure-activity relationships from historical data
- **Competitive intelligence**: See what compounds others have tested

In the next talktorial (AI-PSCI-006), you'll learn to filter these compounds using ADMET and drug-likeness criteria to focus on compounds with the best chance of becoming drugs.

Starting in Week 5, you'll apply these skills to YOUR chosen target from the 6-target portfolio!

---

*AI-PSCI-005 Complete. Proceed to AI-PSCI-006: ADMET Filtering & Drug-Likeness.*